In [66]:
import os 
os.chdir("..")
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

%load_ext autoreload
%autoreload 2


n_qubits = 4
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, noise=noise, problem=problem)

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [188]:
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]
for k in range(vqe_handler.n_qubits):
    indexed_circuit.append(k+vqe_handler.number_of_cnots)
    
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit)

tup=()
aps=0
for ind,k in enumerate(indexed_circuit):
    qubs = vqe_handler.give_qubit(k)
    for iq,q in enumerate(qubs):
        tup+=((ind,cirq.X.on(vqe_handler.qubits[q])),)
tup=((0,cirq.X.on(vqe_handler.qubits[0])),(1,cirq.X.on(vqe_handler.qubits[1])),(1,cirq.X.on(vqe_handler.qubits[2])), (1,cirq.X.on(vqe_handler.qubits[3])))
for tupp in tup:
    circuit.batch_insert(tuple([tupp]))
    
tup=(tuple([(2,cirq.X.on(vqe_handler.qubits[k])) for k in range(4)]))
for tupp in tup:
    circuit.batch_insert(tuple([tupp]))  
    
tup=(tuple([(4,cirq.X.on(vqe_handler.qubits[k])) for k in range(4)]))
for tupp in tup:
    circuit.batch_insert(tuple([tupp]))  
circuit

(0, 0): ───X───Rx(th_0)───X───Rz(th_4)───X───

(0, 1): ───X───Rx(th_1)───X───Rz(th_5)───X───

(0, 2): ───X───Rx(th_2)───X───Rz(th_6)───X───

(0, 3): ───X───Rx(th_3)───X───Rz(th_7)───X───

In [193]:
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]
for k in range(vqe_handler.n_qubits):
    indexed_circuit.append(k+vqe_handler.number_of_cnots)
    
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit)

tup=()
aps=0
for ind,k in enumerate(indexed_circuit):
    qubs = vqe_handler.give_qubit(k)
    if ind < 4:
        ii=0
    else:
        ii=2
    for iq,q in enumerate(qubs):
        tup+=((ii,cirq.X.on(vqe_handler.qubits[q])),)
for tupp in tup:
    circuit.batch_insert(tuple([tupp]))  
circuit

(0, 0): ───X───Rx(th_0)───X───Rz(th_4)───

(0, 1): ───X───Rx(th_1)───X───Rz(th_5)───

(0, 2): ───X───Rx(th_2)───X───Rz(th_6)───

(0, 3): ───X───Rx(th_3)───X───Rz(th_7)───

In [219]:
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]
for k in range(vqe_handler.n_qubits):
    indexed_circuit.append(k+vqe_handler.number_of_cnots)
indexed_circuit = indexed_circuit*4

circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit)

tup=()
aps=0
for ind,k in enumerate(indexed_circuit):
    qubs = vqe_handler.give_qubit(k)
    for iq,q in enumerate(qubs):
        tup+=((ind-int(len(tup)/2)-1,cirq.X.on(vqe_handler.qubits[q])),)
for tupp in tup:
    circuit.batch_insert(tuple([tupp]))  
circuit

(0, 0): ───X───Rx(th_0)───X───Rz(th_4)───X───Rx(th_8)────X───Rz(th_12)───X───Rx(th_16)───X───Rz(th_20)───X───Rx(th_24)───X───Rz(th_28)───

(0, 1): ───X───Rx(th_1)───X───Rz(th_5)───X───Rx(th_9)────X───Rz(th_13)───X───Rx(th_17)───X───Rz(th_21)───X───Rx(th_25)───X───Rz(th_29)───

(0, 2): ───X───Rx(th_2)───X───Rz(th_6)───X───Rx(th_10)───X───Rz(th_14)───X───Rx(th_18)───X───Rz(th_22)───X───Rx(th_26)───X───Rz(th_30)───

(0, 3): ───X───Rx(th_3)───X───Rz(th_7)───X───Rx(th_11)───X───Rz(th_15)───X───Rx(th_19)───X───Rz(th_23)───X───Rx(th_27)───X───Rz(th_31)───

In [177]:
tup[0]

(0, cirq.X(cirq.GridQubit(0, 0)))

In [117]:
circuit.batch_insert(tup)
circuit

(0, 0): ───Rx(th_0)───X───Rz(th_4)───X───────

(0, 1): ───Rx(th_1)───────Rz(th_5)───X───X───

(0, 2): ───Rx(th_2)───────Rz(th_6)───X───X───

(0, 3): ───Rx(th_3)───────Rz(th_7)───X───X───

In [104]:
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]
for k in range(vqe_handler.n_qubits):
    indexed_circuit.append(k+vqe_handler.number_of_cnots)
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit)

circuit

(0, 0): ───Rx(th_0)───Rz(th_4)───

(0, 1): ───Rx(th_1)───Rz(th_5)───

(0, 2): ───Rx(th_2)───Rz(th_6)───

(0, 3): ───Rx(th_3)───Rz(th_7)───

In [108]:
tup = ((0, cirq.X.on(vqe_handler.qubits[0])),)
circuit.batch_insert(tup)
circuit

(0, 0): ───X───Rx(th_0)───Rz(th_4)───

(0, 1): ───────Rx(th_1)───Rz(th_5)───

(0, 2): ───────Rx(th_2)───Rz(th_6)───

(0, 3): ───────Rx(th_3)───Rz(th_7)───

In [114]:
circuitt=circuit.copy()
tup = ((2, cirq.X.on(vqe_handler.qubits[0])),)
circuitt.batch_insert(tup)
circuitt

(0, 0): ───X───Rx(th_0)───X───Rz(th_4)───

(0, 1): ───────Rx(th_1)───────Rz(th_5)───

(0, 2): ───────Rx(th_2)───────Rz(th_6)───

(0, 3): ───────Rx(th_3)───────Rz(th_7)───

In [ ]:
tup=()
aps=0
for ind,k in enumerate(indexed_circuit):
    qubs = vqe_handler.give_qubit(k)
    for iq,q in enumerate(qubs):
        tup+=((ind+len(tup)-1,cirq.X.on(vqe_handler.qubits[q])),)
circuit.batch_insert(tup)
circuit

(0, 0): ───X───Rx(th_0)───────Rz(th_4)───X───────

(0, 1): ───────Rx(th_1)───X───Rz(th_5)───X───────

(0, 2): ───────Rx(th_2)───────Rz(th_6)───X───X───

(0, 3): ───────Rx(th_3)───────Rz(th_7)───X───X───

In [88]:
tup

((0, cirq.X(cirq.GridQubit(0, 0))),
 (1, cirq.X(cirq.GridQubit(0, 1))),
 (2, cirq.X(cirq.GridQubit(0, 2))),
 (3, cirq.X(cirq.GridQubit(0, 3))),
 (4, cirq.X(cirq.GridQubit(0, 0))),
 (5, cirq.X(cirq.GridQubit(0, 1))),
 (6, cirq.X(cirq.GridQubit(0, 2))),
 (7, cirq.X(cirq.GridQubit(0, 3))))

In [47]:
circuit

(0, 0): ───Rx(th_0)───Rz(th_4)───

(0, 1): ───Rx(th_1)───Rz(th_5)───

(0, 2): ───Rx(th_2)───Rz(th_6)───

(0, 3): ───Rx(th_3)───Rz(th_7)───

In [54]:
its = (([[2, cirq.I.on(vqe_handler.qubits[0])]]))
circuit.batch_insert(its)
circuit

┌──┐
(0, 0): ───Rx(th_0)───@───I───@───@───X─────────X────I───I───
                      │       │   │   │         │
(0, 1): ───Rx(th_1)───X───────┼───┼───@───@────@┼────────X───
                              │   │       │    ││        │
(0, 2): ───Rx(th_2)───────────X───┼───────X────┼@────────@───
                                  │            │
(0, 3): ───Rx(th_3)───────────────X────────────X─────────────
                                              └──┘

In [ ]:
def noisy_batch(circuit,args={"channel":"depolarizing", "p":0.1}):
    if "channel" == "depolarizing":
        pauls = [cirq.X, cirq.Y, cirq.Z]
        #circuit_res = cirq.resolve_parameters(circuit, {s:0 for s,k in zip(symbols, range(len(symbols)))})
        p=0.01
        cbatch = []
        totals=10**3
        for k in range(totals):
            circ = circuit.copy()
            if k<int((1-p)*totals):
                cbatch.append(circ)
            elif int((1-p)*totals)<=k:
                which = np.random.choice(range(3), 1)[0]
                gate = pauls[which]
                for q in vqe_handler.qubits:
                    circ.append(gate.on(q))
                cbatch.append(circ)

In [ ]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(circuit_input, symbol_names=symbols, operators=tfq.convert_to_tensor([vqe_handler.observable]*1000),
        initializer=tf.keras.initializers.RandomUniform(minval=-np.pi, maxval=np.pi))

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=vqe_handler.lr)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor(cbatch)
qoutput = tf.ones((10**3, 1))*vqe_handler.lower_bound_Eg
from datetime import datetime
now = datetime.now()
h=model.fit(x=tfqcircuit, y=qoutput, batch_size=10**3, epochs=100,
          verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=vqe_handler.patience, mode="min", min_delta=10**-3)])
print(datetime.now()-now)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(h.history["loss"])